<a href="https://colab.research.google.com/github/LaizaCostaa/Matching-de-curriculos/blob/main/processamento_curriculos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEITURA DOS ARQUIVOS PDF (Currículos)

In [ ]:
 pip install fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.9-py2.py3-none-any.whl size=35614 sha256=c1c4fa27345cfb6c03fea1cba06c912f39ca4630447a73dc652c29400b27b781
  Stored in directory: /root/.cache/pip/wheels/a1/6c/03/6c5e3cf1a6e4b9e

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 27.2 MB/s eta 0:00:00


In [ ]:
pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import shutil
import os
import fnmatch
import datetime

#from pypdf import PdfReader
from openai import OpenAI

import fitz  # PyMuPDF
from google.colab import drive

# Bibliotecas para escrever nas planilhas
from google.colab import auth
import gspread
from gspread_dataframe import get_as_dataframe
from google.auth import default

In [ ]:
# Montar o Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:


# Montar o Google Drive
drive.mount('/content/drive')

def get_id_candidata():
  with open('/content/drive/MyDrive/Portfólio/projeto_recrutamento/last_id_candidata.txt') as file:
    last_id_candidata = file.read()
    last_id_candidata = int(last_id_candidata)
    id_candidata = last_id_candidata + 1

  return id_candidata


def update_id_candidata(id_candidata):
  with open('/content/drive/MyDrive/Portfólio/projeto_recrutamento/last_id_candidata.txt', 'w') as file:
    file.write(str(id_candidata))



def list_pdf_files(directory):
  """
  List all PDF files in the specified directory.

  Args:
  - directory (str): Path to the directory to search.

  Returns:
  - list: List of PDF files found in the directory.
  """
  pdf_files = []
  for root, dirs, files in os.walk(directory):
    for filename in files:
        if fnmatch.fnmatch(filename, '*.pdf'):
            pdf_files.append(os.path.join(root, filename))
  return pdf_files


'''
def read_pdf(pdf_path):
    reader = PdfReader(pdf_path)

    pdf_text = ''
    for page in reader.pages:
        pdf_text += page.extract_text()

    return pdf_text
'''


def extract_text_from_pdf(pdf_path):
  # Abrir o arquivo PDF
  document = fitz.open(pdf_path)

  # Extrair texto de todas as páginas
  text = ""
  for page_num in range(len(document)):
    page = document.load_page(page_num)
    text += page.get_text()

  # Fechar o documento
  document.close()

  return text


def get_prompt():
  prompt = f"""
          Leia o curriculo e me gere várias tabelas, no formato csv seperados por pipe e sem header, com seus respectivos dados para que eu possa alimentar essa planilha. Utilize como id_candidata o valor {str(id_candidata)}. Onde tiver o campo nome_candita, coloque o nome completo da candidata. As abas das planilhas em aenxo sao as seguintes:

          Tabela candidatas. Campos: id_candidata, nome_candidata, cargo_candidata, email, telefone, cidade, estado, linkedin
          Tabela resumo. Campos: id_candidata, resumo, palavras_chave
          Tabela educacao. Campos: id_candidata, grau, curso, instituicao
          Tabela experiencia. Campos: id_candidata, empresa_experiencia, cargo_experiencia, inicio, termino, tempo
          Tabela competencias_curriculo. Campos: id_candidata, nome_candidata, competencias, tipo

          Evite excesso de informação, como instituição, data (exceto para a experiencia do cargo) e o que mais achar desnecessário. Apenas alimente as informacoes que sao pertinentes as tabelas e suas colunas. Pretensao salarial, jornada de trabalho, etc sao exemplos que nao serao inseridos nas tabelas. ESTES dados serao usados para montar um dashboard, entao organize de forma que fique uma informacao em cada linha, repitindo o id quando precisar, por exemplo, se a candidata possui vários cursos de capacitacao, repita o id dela para mencionar cada curso em uma linha. na coluna cargo_candidata, caso náo haja a profissao ou cargo da candidata bem definido, escreva o objetivo de cargo que ela busca. Procure pelo tópico ``objetivo`` no curriculo. Na tabela educacao preencha com a formacao academica da pessoa, incluindo tambem a instituição. CLASSIFIQUE O TIPO DA COMPETENCIA, SE É UMA HABILIDADE, UM CURSO, UMA CERTIFICACO ETC. caso nao tenha um resumo no curriculo, elabore um, com base nos dados do curriculo. Pule a parte de preencher a coluna curriculo_drive
          Na tabela candidatas, no campo linkedin, retorne o hyperlink para a pagina do linkedin da candidata. Se não encontrar o link, apenas deixe em branco este campo.

          Nao retorne informações adicionais na sua resposta, apenas retorne o conteudo de cada tabela no seguinte formato:
          Tabela <nome_da_tabela>: <conteudo_campo_1>|<conteudo_campo_2>|<...>|<conteudo_campo_n>
          Conteudo do curriculo:
          """
  return prompt


def set_environment_vars():
  with open('/content/drive/MyDrive/Portfólio/projeto_recrutamento/open_ai_key.txt') as file:
    OPENAI_API_KEY = file.read()
    # Definir uma variável de ambiente
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
  return None


def send_openai_request(prompt):
  client = OpenAI()

  completion = client.chat.completions.create(
    model="gpt-4o",
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt},

    ]
  )
  return completion
  # print(completion.choices[0].message)


def trata_response(response):
  # Constantes
  TABLE_NAME_POSITION = 0
  TABLE_CONTENT_POSITION = 1
  DELIMITER_TABLE_AND_CONTENT = ':'
  DELIMITER_END_CONTENT = '\n'
  DELIMITER_END_OF_RESPONSE = '```'

  tables_response = response.split('Tabela ')[1:]

  tabelas_dict = dict()
  for table_response in tables_response:
    table_response =  table_response.split(DELIMITER_TABLE_AND_CONTENT)
    table_response[1] = table_response[1].replace(DELIMITER_END_CONTENT, '')
    table_response[1] = table_response[1].replace(DELIMITER_END_OF_RESPONSE, '')

    # Se a tabela ainda nao existir no dicionario, cria uma nova chave {<nome_tabela>: <list(conteudo_tabela)>}
    if table_response[TABLE_NAME_POSITION] not in tabelas_dict:
        tabelas_dict[table_response[TABLE_NAME_POSITION]] = [table_response[TABLE_CONTENT_POSITION]]
    # Se a tabela ja existir no dicionario, apenas appenda com o conteudo anterior
    else:
        tabelas_dict[table_response[TABLE_NAME_POSITION]] = tabelas_dict[table_response[TABLE_NAME_POSITION]] + [table_response[TABLE_CONTENT_POSITION]]

  return tabelas_dict


def write_on_spreadsheet(tabelas_dict, spreadsheet_id):
  # Autenticar
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)

  try:
    # Abrir a planilha
    sh = gc.open_by_key(spreadsheet_id)

    for worksheet_tab, table_content_list in tabelas_dict.items():
      # Selecionar a aba da planilha
      worksheet = sh.worksheet(worksheet_tab)

      # Encontrar a última linha com dados
      last_row = len(worksheet.get_all_values()) + 1  # +1 para a nova linha

      table_new_data = []
      for table_content in table_content_list:
        table_new_data.append(table_content.strip().split('|'))
        # Dados a serem inseridos
        new_data = table_new_data  # formato de new_data: [[]] (lista de listas). Cada elemento da lista é uma celula da planilha

        #print(new_data)
      # Inserir dados na próxima linha disponível
      worksheet.insert_rows(new_data, row=last_row)

  except gspread.SpreadsheetNotFound:
    print("Erro: Planilha não encontrada. Verifique o ID da planilha e as permissões.")
  except gspread.WorksheetNotFound:
    print(f"Erro: Aba '{worksheet_tab}' não encontrada. Verifique o nome da aba.")
  except Exception as e:
    print(f"Erro: {e}")


def move_cv_to_processed(cv_name, curriculos_path):
  # Apos o processamento do curriculo, move o arquivo para a pasta de processados
  origem = f'{curriculos_path}/nao_processados/{cv_name}'
  destino = f'{curriculos_path}/processados/{cv_name}'

  try:
      shutil.move(origem, destino)
      print(f'Arquivo movido de {origem} para {destino} com sucesso.')
  except FileNotFoundError:
      print(f'Erro: O arquivo {origem} não foi encontrado.')
  except PermissionError:
      print(f'Erro: Permissão negada para mover o arquivo {origem}.')
  except Exception as e:
      print(f'Ocorreu um erro ao mover o arquivo: {str(e)}')


def get_path_cvs_origin(cv_origin):
  # Retorna o caminho do diretorio dos curriculos nao processados
  if cv_origin == 'curriculos':
      curriculos_path = '/content/drive/MyDrive/Portfólio/projeto_recrutamento/curriculos'   # curriculos

  return curriculos_path


def move_arquivos(origem, destino):
  try:
      # Verifica se o diretório de destino existe; se não, cria-o
      if not os.path.exists(destino):
          os.makedirs(destino)

      # Lista todos os arquivos na pasta de origem
      arquivos = os.listdir(origem)

      for arquivo in arquivos:
          if arquivo.endswith('.csv'):
              caminho_origem = os.path.join(origem, arquivo)
              caminho_destino = os.path.join(destino, arquivo)

              # Move o arquivo para o diretório de destino
              shutil.move(caminho_origem, caminho_destino)
              print(f"Arquivo '{arquivo}' movido para '{destino}'")

      print("Todos os arquivos foram movidos com sucesso!")

  except Exception as e:
      print(f"Ocorreu um erro ao mover os arquivos: {e}")



if __name__ == '__main__':
  print('Inicio do programa')

  # Seta variaveis de ambiente
  set_environment_vars()

  # Constantes
  cv_origins = ['curriculos']  # Origem dos curriculos
  spreadsheet_id = '1xce02R3RNaC1fvfaRCr--dZ-CnwFw0Cxb8WYCZebHtI'  # ID da  planilha

  for cv_origin in cv_origins:
    curriculos_path = get_path_cvs_origin(cv_origin)

    # Lista todos os curriculos em pdf
    pdf_files = list_pdf_files(f'{curriculos_path}/nao_processados')  # retorna uma lista de paths dos curriculos nao processados

    for pdf_file in pdf_files:
      id_candidata = get_id_candidata()  # Gera o id da candidata

      # Le o texto do curriculo
      pdf_text = extract_text_from_pdf(pdf_file)
      cv_name = pdf_file.split('/')[-1]  # Pega o nome do curriculo

      # Constroi o prompt e concatena com o conteudo do curriculo
      prompt = get_prompt()
      prompt = prompt + pdf_text

      # Manda requisicao pra API do chatgpt
      completion = send_openai_request(prompt)
      print(completion.choices[0].message)

      # Trata resposta do chatgpt e retorna as tabelas num dicionario {nome_da_tabela: conteudo_da_tabela}
      tabelas_dict = trata_response(completion.choices[0].message.content)

      # Escreve os dados no csv
      write_on_spreadsheet(tabelas_dict, spreadsheet_id)

      update_id_candidata(id_candidata)

      # Apos o processamento do curriculo, move o arquivo para a pasta de processados
      move_cv_to_processed(cv_name, curriculos_path)

Mounted at /content/drive
Inicio do programa
ChatCompletionMessage(content='Tabela candidatas: 1151|Brasileira, casada, residente em São Paulo / SP|GESTÃO DE COMPRAS | SUPRIMENTOS|contato@outlierscareers.com.br|(11) 2737-1724|São Paulo|SP|\nTabela resumo: 1151|Profissional com sólida carreira desenvolvida em empresas multinacionais de médio e grande porte dos segmentos de telecom, financeiro, indústria de vidros, construção e montadora de veículos; Expertise em gestão de compras e suprimentos, comprometida com o negócio, com grandes realizações e contribuindo ativamente para o alcance dos propósitos da corporação; Gestão e desenvolvimento de equipes de alta performance, compartilhando decisões e responsabilidades, alinhadas com as metas e objetivos da organização; Atuação efetiva na condução e gestão de projetos, compras recorrentes e pontuais para categorias diretas e indiretas: TI, Engenharia, Infraestrutura, Facilities, Marketing, RH e Call Center; Competência em planejamento estrat